<a href="https://colab.research.google.com/github/OmarAmeer96/Wheel-n-Deal/blob/AI/recommendationsys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357245 sha256=7ed2da879d3729213450b1223ec36665af86be70f170d71a1ea0a4e734609925
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


#1.Load and Preprocess the Data

In [8]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic

# Load the data
users_df = pd.read_csv('users.csv')
commuters_df = pd.read_csv('commuters.csv')

# Merge users and commuters on city
merged_df = pd.merge(users_df, commuters_df, on='city')

# Add a dummy rating column
merged_df['rating'] = 1

# Define a reader to read the dataset
reader = Reader(rating_scale=(1, 1))

# Load the data into a Surprise dataset
data = Dataset.load_from_df(merged_df[['user_id', 'commuter_id', 'rating']], reader)


In [9]:
users_df.head()

,user_id,user_name,city
0,1,Alice,New York
1,2,Bob,Los Angeles
2,3,Charlie,Chicago
3,4,Diana,New York
4,5,Eve,Los Angeles


In [10]:
commuters_df.head()

,commuter_id,commuter_name,city
0,1,John Doe,New York
1,2,Jane Smith,Los Angeles
2,3,Emily Davis,New York
3,4,Michael Brown,Chicago
4,5,Sarah Wilson,New York


#2. Train-Test **Split**

In [ ]:
# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25)


#3. Model Training


In [ ]:
# Use the KNNBasic algorithm
algo = KNNBasic(k=3)

# Train the algorithm on the training set
algo.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


#4. Model Testing and Evaluation


In [ ]:
from surprise import accuracy

# Predict ratings for the test set
predictions = algo.test(testset)

# Calculate and print the RMSE
accuracy.rmse(predictions)


RMSE: 0.0000


0.0

#5. Making Recommendations
To recommend commuters based on user preferences:

In [ ]:
def recommend_commuters_for_user(user_id, users_df, commuters_df, algo, top_n=10):
    # Get the user's city
    user_city = users_df[users_df['user_id'] == user_id]['city'].values[0]

    # Filter commuters from the same city
    same_city_commuters = commuters_df[commuters_df['city'] == user_city]

    # Predict ratings for all commuters in the same city
    same_city_commuters['predicted_rating'] = same_city_commuters['commuter_id'].apply(
        lambda x: algo.predict(user_id, x).est
    )

    # Sort by predicted rating and return the top N commuters
    top_commuters = same_city_commuters.sort_values(by='predicted_rating', ascending=False).head(top_n)

    return top_commuters

# Example usage
user_id = 1
recommended_commuters = recommend_commuters_for_user(user_id, users_df, commuters_df, algo)
print(recommended_commuters)


   commuter_id    commuter_name      city  predicted_rating
0            1         John Doe  New York                 1
2            3      Emily Davis  New York                 1
4            5     Sarah Wilson  New York                 1
7            8  Patricia Taylor  New York                 1


<ipython-input-7-932d40eeb2b9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  same_city_commuters['predicted_rating'] = same_city_commuters['commuter_id'].apply(
